<a href="https://colab.research.google.com/github/Eragon424/speech-tonality-analysis/blob/main/classification_(7).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install scikit-learn

Note: you may need to restart the kernel to use updated packages.


In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

import librosa
import os
import pathlib

import pylab as pl
import scipy.optimize as opt
from sklearn import preprocessing
from sklearn.model_selection import train_test_split

import csv

In [ ]:
#датасет  не с балансируемым полным набором тишина удалена
data_x_train = pd.read_csv('dataset__balance_no_silince_train.csv')
data_y_train = pd.read_csv('y_train_full')
data_x_train['label']= data_y_train['annotator_emo']


#удалим эмоции с пометкой другие
grouped = data_x_train
data_x_train = grouped[grouped.label != 'other' ]

#сделаем новые метки классов тональности angry = -1 neutral = 0  positive =1  sad -1
data_x_train.loc[(data_x_train.label =='angry'),('label')] = 1
data_x_train.loc[(data_x_train.label =='neutral'),('label')] = 0
data_x_train.loc[(data_x_train.label =='positive'),('label')] = 2
data_x_train.loc[(data_x_train.label =='sad'),('label')] = 1
data_x_train['label'].astype(int)

data_x_train

,filename,mfcc_mean1,mfcc_mean2,mfcc_mean3,mfcc_mean4,mfcc_mean5,mfcc_mean6,mfcc_mean7,mfcc_mean8,mfcc_mean9,...,mfcc_std13,mfcc_std14,mfcc_std15,mfcc_std16,mfcc_std17,mfcc_std18,mfcc_std19,mfcc_std20,cent_mean,label
0,00000d522439136554c888f4cfd92131.wav,-467.08884,67.734410,13.555082,38.741455,-7.167416,0.597688,-20.886284,15.138917,-8.795949,...,8.588613,8.318792,6.618607,7.076600,7.222637,7.806918,5.388669,7.536538,wavs,0
1,000039c2bc753aa5a776621a4707eb73.wav,-455.32877,69.964820,-28.523523,32.344345,-21.611555,-1.738283,-29.974870,-2.658362,-27.071587,...,7.017729,5.085210,5.052448,5.966898,4.432629,5.499484,4.201167,8.535248,wavs,0
2,000043e573826e21659ef9401d4ba84f.wav,-324.05480,115.785450,-21.114456,35.512660,-13.603605,-4.507683,-6.577841,-9.184050,-18.723753,...,18.828270,9.352553,10.229453,9.870971,6.685668,7.650016,10.031831,6.412236,wavs,0
3,000062a9bd8daf199cc520508902b304.wav,-412.43720,120.690140,4.870650,44.868004,-22.577837,15.440735,-18.214250,3.675842,-6.553605,...,6.921843,4.515031,5.963138,6.237720,5.724922,4.901245,5.649538,4.081949,wavs,0
4,000182128001302626efa6032d095cc8.wav,-477.00810,113.069885,-34.195324,34.832405,-21.345510,25.681898,-27.018713,4.594828,-17.452380,...,7.554842,7.380126,8.139834,7.889399,5.080637,6.300990,5.740642,5.134631,wavs,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
184628,fffdcb6c9078c84c88d65752474c8945.wav,-406.72284,103.964610,-42.010303,60.079086,-42.139908,36.664444,-42.244087,19.113665,-19.777035,...,5.468694,5.583583,5.220826,6.555201,5.548795,5.608561,5.275757,4.273073,wavs,1
184629,fffe6e2189d87d2bcf4e10fba9a4aa73.wav,-310.43698,161.317870,-11.443884,39.847830,13.334776,28.219568,7.392763,11.241220,-1.343260,...,7.814247,11.815525,8.904210,9.721940,6.524000,6.089618,7.029152,5.702905,wavs,0
184630,fffeba24094a8cdf1737387158a8b220.wav,-266.47668,124.357830,-9.233796,25.878456,-13.423601,-8.512863,-30.396950,1.176547,-5.997843,...,10.373017,6.366199,7.475891,10.202706,7.864334,9.025516,6.211788,5.641520,wavs,0
184631,fffeefeeb5cc057d212d08dab33b9b56.wav,-385.38248,154.218920,-13.083211,22.870913,14.446136,26.102562,-9.495589,8.869211,-1.466560,...,7.554702,9.499482,9.867852,6.452058,7.535012,8.333311,6.215310,6.899868,wavs,0


Разделим наши данные на зависимую (целевую) - (у) переменную и независимые -(х) переменные.
Классификация пытается узнать связь между набором переменных характеристик (х) и целевой переменной (у)
Целевой атрибут в классификации является категориальной переменной в колонке 'label' - это метка (правильный ответ). Учитывая набор обучающих точек данных вместе с целевыми метками, классификация определяет метку класса для немаркированного тестового случая.
Цель прогноза классификация тональности (эмоций).
Классификация заключается в использовании существующих данных, которые имеют информацию об аудиосигнале - значимые характеристики аудиоданных (45 значений), для создания классификатора, который смог бы использовать этот опыт для решения той же проблемы в новых ситуациях, и на новых данных, а затем и маркировки новых данных.
Метод - value_counts(). Подсчет значений - это хороший способ понять, сколько единиц каждой характеристики / переменной у нас есть.
Посмотрим на подсчет значений в колонке 'label'.

In [ ]:
data_x_train['label'].value_counts()

0    109186
1     53636
2     19930
Name: label, dtype: int64

y - зависимая (целевая) переменная это колонка 'label' в нашем наборе данных
Чтобы использовать библиотеку scikit-learn, нам нужно преобразовать Dataframes (фрейм данных) Pandas в массив Numpy.

In [ ]:
y = data_x_train['label'].values
y=y.astype('int')
y.value_counts()


AttributeError: 'numpy.ndarray' object has no attribute 'value_counts'

In [ ]:
#Посмотрим на название колонок в наборе данных:
data_x_train.columns

Index(['filename', 'mfcc_mean1', 'mfcc_mean2', 'mfcc_mean3', 'mfcc_mean4',
       'mfcc_mean5', 'mfcc_mean6', 'mfcc_mean7', 'mfcc_mean8', 'mfcc_mean9',
       'mfcc_mean10', 'mfcc_mean11', 'mfcc_mean12', 'mfcc_mean13',
       'mfcc_mean14', 'mfcc_mean15', 'mfcc_mean16', 'mfcc_mean17',
       'mfcc_mean18', 'mfcc_mean19', 'mfcc_mean20', 'mfcc_std1', 'mfcc_std2',
       'mfcc_std3', 'mfcc_std4', 'mfcc_std5', 'mfcc_std6', 'mfcc_std7',
       'mfcc_std8', 'mfcc_std9', 'mfcc_std10', 'mfcc_std11', 'mfcc_std12',
       'mfcc_std13', 'mfcc_std14', 'mfcc_std15', 'mfcc_std16', 'mfcc_std17',
       'mfcc_std18', 'mfcc_std19', 'mfcc_std20', 'cent_mean', 'label'],
      dtype='object')

In [ ]:
# в столбце cent_mean есть NAN значения, их нужно убрать, чтобы обучать модель используем метод fillna
data_x_train=data_x_train.fillna(method='ffill')

X - независимые переменные. Это значимые характеристики аудиоданных (45 значений)

X = data_x_train[['mfcc_mean1', 'mfcc_mean2', 'mfcc_mean3', 'mfcc_mean4',
       'mfcc_mean5', 'mfcc_mean6', 'mfcc_mean7', 'mfcc_mean8', 'mfcc_mean9',
       'mfcc_mean10', 'mfcc_mean11', 'mfcc_mean12', 'mfcc_mean13',
       'mfcc_mean14', 'mfcc_mean15', 'mfcc_mean16', 'mfcc_mean17',
       'mfcc_mean18', 'mfcc_mean19', 'mfcc_mean20', 'mfcc_std1', 'mfcc_std2',
       'mfcc_std3', 'mfcc_std4', 'mfcc_std5', 'mfcc_std6', 'mfcc_std7',
       'mfcc_std8', 'mfcc_std9', 'mfcc_std10', 'mfcc_std11', 'mfcc_std12',
       'mfcc_std13', 'mfcc_std14', 'mfcc_std15', 'mfcc_std16', 'mfcc_std17',
       'mfcc_std18', 'mfcc_std19', 'mfcc_std20', 'cent_mean', 'cent_std',
       'cent_skew', 'rolloff_mean', 'rolloff_std']]
X[0:5]

In [ ]:
X = data_x_train[['mfcc_mean1', 'mfcc_mean2', 'mfcc_mean3', 'mfcc_mean4',
       'mfcc_mean5', 'mfcc_mean6', 'mfcc_mean7', 'mfcc_mean8', 'mfcc_mean9',
       'mfcc_mean10', 'mfcc_mean11', 'mfcc_mean12', 'mfcc_mean13',
       'mfcc_mean14', 'mfcc_mean15', 'mfcc_mean16', 'mfcc_mean17',
       'mfcc_mean18', 'mfcc_mean19', 'mfcc_mean20', 'mfcc_std1', 'mfcc_std2',
       'mfcc_std3', 'mfcc_std4', 'mfcc_std5', 'mfcc_std6', 'mfcc_std7',
       'mfcc_std8', 'mfcc_std9', 'mfcc_std10', 'mfcc_std11', 'mfcc_std12',
       'mfcc_std13', 'mfcc_std14', 'mfcc_std15', 'mfcc_std16', 'mfcc_std17',
       'mfcc_std18', 'mfcc_std19', 'mfcc_std20']]
X[0:5]

,mfcc_mean1,mfcc_mean2,mfcc_mean3,mfcc_mean4,mfcc_mean5,mfcc_mean6,mfcc_mean7,mfcc_mean8,mfcc_mean9,mfcc_mean10,...,mfcc_std11,mfcc_std12,mfcc_std13,mfcc_std14,mfcc_std15,mfcc_std16,mfcc_std17,mfcc_std18,mfcc_std19,mfcc_std20
0,-467.08884,67.734410,13.555082,38.741455,-7.167416,0.597688,-20.886284,15.138917,-8.795949,-4.386859,...,8.111265,9.971217,8.588613,8.318792,6.618607,7.076600,7.222637,7.806918,5.388669,7.536538
1,-455.32877,69.964820,-28.523523,32.344345,-21.611555,-1.738283,-29.974870,-2.658362,-27.071587,-14.633931,...,6.962493,6.895723,7.017729,5.085210,5.052448,5.966898,4.432629,5.499484,4.201167,8.535248
2,-324.05480,115.785450,-21.114456,35.512660,-13.603605,-4.507683,-6.577841,-9.184050,-18.723753,-15.142766,...,11.886771,13.955037,18.828270,9.352553,10.229453,9.870971,6.685668,7.650016,10.031831,6.412236
3,-412.43720,120.690140,4.870650,44.868004,-22.577837,15.440735,-18.214250,3.675842,-6.553605,-7.462927,...,8.359876,8.249008,6.921843,4.515031,5.963138,6.237720,5.724922,4.901245,5.649538,4.081949
4,-477.00810,113.069885,-34.195324,34.832405,-21.345510,25.681898,-27.018713,4.594828,-17.452380,1.749368,...,10.516548,7.178211,7.554842,7.380126,8.139834,7.889399,5.080637,6.300990,5.740642,5.134631


X также нужно преобразовать Dataframes (фрейм данных) Pandas в массив Numpy.

Стандартизации набора данных
Если в наборе данных есть некоторые выбросы или значения переменных находятся в разных диапазонах, необходимо исправить их.

Стандартизация данных дает нулевое среднее значение и единичную дисперсию,

Пакет предварительной обработки SciKit Learn предоставляет несколько общих служебных функций и классов трансформаторов для изменения необработанных векторных объектов в подходящую форму вектора для моделирования.

In [ ]:
from sklearn import preprocessing
X= preprocessing.StandardScaler().fit(X).transform(X)
X[0:2]

array([[-0.38176882, -1.51906071,  1.19722768,  0.35407143,  0.61474435,
        -1.29791423, -0.51302425,  1.01642758,  0.09140203, -0.62130851,
         0.30533685,  0.45507495,  0.56449373,  0.56218751,  0.16863973,
         0.85506944, -0.27322038,  0.28678452,  0.56558125, -0.08313971,
         0.94504759,  0.75685573,  0.68380159,  0.5169614 ,  0.7567169 ,
         1.52633272,  2.19961267,  1.32607917,  0.75365158,  1.53740976,
        -0.22797152,  0.9602555 ,  0.55275249,  0.34535876, -0.03566159,
        -0.4745071 ,  0.60562956,  0.52261493, -0.34072877,  0.79779667],
       [-0.24018679, -1.42117499, -1.37030587, -0.10689177, -0.46940315,
        -1.49541522, -1.27234934, -0.95820371, -2.0741589 , -2.27480679,
        -1.5792532 , -2.3340465 , -1.75464524, -1.55120048, -1.77703525,
        -1.19324997, -1.60792188, -1.25517892, -1.04643523, -1.88693192,
         0.43007941, -0.32421754, -0.14484971,  0.39673959, -0.91440392,
        -0.05929297,  0.88275978, -0.78666412,  0.

Разделение набора данных на тренировочные и тестовые наборы (train/test split) и настройка алгоритма.

Точность вне выборки - это процент правильных прогнозов, которые модель делает для данных, на которых НЕ была обучена модель. Выполнение обучения и тестирования одного и того же набора данных, скорее всего, будет иметь низкую точность вне выборки из-за вероятности переобучения нашей модели.

Важно, чтобы наши модели имели высокую точность вне выборки, потому что цель любой модели, конечно, состоит в том, чтобы делать правильные прогнозы на неизвестных данных. Итак, как мы можем улучшить точность вне выборки? Один из способов - использовать метод оценки, называемый разделением train/test split.

Разделение на обучение / тестирование включает в себя разделение набора данных на обучающий и тестовый наборы соответственно, которые являются взаимоисключающими. После этого тренируем с обучающим набором (train) и тестируем с помощью набора для тестирования (test).

Это обеспечит более точную оценку точности вне выборки, поскольку набор данных тестирования (test) не является частью набора данных, который использовался для обучения модели (train).

Для анализа необходимо разделить набор данных на тренировочные и тестовые наборы (train/test split).

test_size представляет собой соотношение тестового набора данных, а random_state гарантирует, что мы получим те же разбиения.

Для преобразования данных в более высоком пространственном пространстве используется Математическая функция, известная, как функция ядра - RBF (радиальная базисная функция)

Размер тренировочных и тестовых наборов (количество строк, колонок)
Прогнозирование новых значений в тестовом наборе после обучения на тренировочном наборе.

In [ ]:
from sklearn.model_selection import train_test_split
# Разделение набора данных на тренировочные и тестовые наборы (train/test split)
X_train, X_test, y_train, y_test = train_test_split( X, y, test_size=0.3,
                                                            random_state=20)

# размер тренировочных и тестовых наборов (количество строк, колонок):
print ('Train set:', X_train.shape, y_train.shape)
print ('Test set:', X_test.shape, y_test.shape)

from sklearn import svm
clf = svm.SVC(kernel='rbf')   #  функция ядра - RBF (радиальная базисная функция)
clf.fit(X_train, y_train)     # Обучение модели на тренировочном наборе
yhat = clf.predict(X_test)    # для прогнозирования новых значений:
yhat [0:10]

Train set: (127926, 40) (127926,)
Test set: (54826, 40) (54826,)


array([0, 0, 0, 0, 0, 0, 1, 1, 0, 0])

Сравнение реальных значений и новых значений (после прогнозирования) - визуально сравнить прогнозы с фактическими значениями.

In [ ]:
print("Prediction:", yhat[0:20])
print("Real Value:", y_test[0:20])

Prediction: [0 0 0 0 0 0 1 1 0 0 0 0 0 0 0 0 1 0 0 0]
Real Value: [1 0 0 0 0 0 0 1 0 0 1 2 0 0 1 1 1 0 2 0]


Метрики для оценки точности модели
Показатели оценки объясняют эффективность модели.  Мы обучили модель, и теперь хотим рассчитать ее точность с помощью тестового набора.

Импортируем метрики из sklearn и проверим точность нашей модели. В основном, мы сравниваем фактические значения в тестовом наборе со значениями, предсказанными моделью, чтобы рассчитать точность модели.  Существуют различные метрики оценки моделей

      - оценка классификации точности - это функция, которая вычисляет точность подмножества, долю правильных ответов алгоритма. По сути, он вычисляет, насколько точно совпадают фактические и прогнозируемые метки в тестовом наборе.
Если весь набор предсказанных меток для выборки строго соответствует истинному набору меток, то точность подмножества равна 1,0 , в противном случае 0,0.

In [ ]:
# accuracy evaluation
from sklearn import metrics
print("Train set Accuracy: ", metrics.accuracy_score(y_train, clf.predict(X_train)))
print("Test set Accuracy: ",metrics.accuracy_score(y_test, yhat) )

Train set Accuracy:  0.6842940449947627
Test set Accuracy:  0.6622587823295517


F1 score - Сonfusion matrix (матрица путаницы, матрица ошибок)
Другой способ проверить точность классификатора - это посмотреть на матрицу неточностей.

Эта матрица показывает корректные и неправильные прогнозы по сравнению с фактическими метками.

Каждая строка матрицы путаницы показывает метки Actual/True в тестовом наборе, а столбцы показывают прогнозируемые метки по классификатору. Мы можем интерпретировать эти числа как количество истинных положительных, ложных отрицательных, истинных отрицательных и ложных срабатываний.  Матрица путаницы показывает способность модели правильно предсказать или разделить классы.

Основываясь на количестве в каждой секции, мы можем рассчитать precision (точность) и Recall (отзывчивость) каждой метки:
Precision (Точность) - это мера точности при условии, что метка класса была предсказана. Он определяется как:
Recall (Отзыв) - это истинно положительный показатель. Это определяется как: Recall (Отзыв) = TP / (TP + FN)
Итак, мы можем вычислить Precision ( точность) и Recall (отзыв) каждого класса.
F1 score: Теперь мы можем рассчитать оценки F1 для каждой метки на основе Precision и Recall этой метки.
F1 score представляет собой гармоническое среднее значение Precision (точности) и Recall (отзывчивости), где F1 score (оценка F1) достигает своего лучшего значения при 1 (идеальная точность и отзывчивость) и худшего при 0.

Это хороший способ показать, что классификатор имеет хорошее значение для Recall (отзыв) и Precision (точность).

Наконец, мы можем сказать, что средняя Accuracy (точность) для этого классификатора - это средний балл F1 для обеих меток,


In [ ]:
from sklearn.metrics import classification_report,confusion_matrix

print('матрица ошибок :\n')
print(confusion_matrix(y_test,yhat))
print('\n')
print('Отчёт:\n')
print(classification_report(y_test,yhat))

матрица ошибок :

[[31509  1114   298]
 [11853  3640   368]
 [ 4079   805  1160]]


Отчёт:

              precision    recall  f1-score   support

           0       0.66      0.96      0.78     32921
           1       0.65      0.23      0.34     15861
           2       0.64      0.19      0.29      6044

    accuracy                           0.66     54826
   macro avg       0.65      0.46      0.47     54826
weighted avg       0.66      0.66      0.60     54826



In [ ]:
from sklearn import tree
model=tree.DecisionTreeClassifier(criterion="entropy")
model.fit(X_train, y_train)

DecisionTreeClassifier(criterion='entropy')

In [ ]:
yhat_tree = model.predict(X_test)

In [ ]:
print('CONFUSION_MATRIX :\n')
print(confusion_matrix(y_test,yhat_tree))
print('\n')
print('REPORT :\n')
print(classification_report(y_test,yhat_tree))

CONFUSION_MATRIX :

[[21285  8779  2857]
 [ 8216  5783  1862]
 [ 2724  1790  1530]]


REPORT :

              precision    recall  f1-score   support

           0       0.66      0.65      0.65     32921
           1       0.35      0.36      0.36     15861
           2       0.24      0.25      0.25      6044

    accuracy                           0.52     54826
   macro avg       0.42      0.42      0.42     54826
weighted avg       0.53      0.52      0.52     54826



In [ ]:
clf_noparams = svm.SVC()   #  функция ядра
clf_noparams.fit(X_train, y_train)     # Обучение модели на тренировочном наборе
yhat_noparams = clf_noparams.predict(X_test)

In [ ]:
print('матрица ошибок :\n')
print(confusion_matrix(y_test,yhat_noparams))
print('\n')
print('Отчёт:\n')
print(classification_report(y_test,yhat_noparams))

матрица ошибок :

[[31509  1114   298]
 [11853  3640   368]
 [ 4079   805  1160]]


Отчёт:

              precision    recall  f1-score   support

           0       0.66      0.96      0.78     32921
           1       0.65      0.23      0.34     15861
           2       0.64      0.19      0.29      6044

    accuracy                           0.66     54826
   macro avg       0.65      0.46      0.47     54826
weighted avg       0.66      0.66      0.60     54826



In [ ]:

from sklearn.neighbors import KNeighborsClassifier
model_km = KNeighborsClassifier(n_neighbors = 1)
model_km.fit(X_train, y_train)
yhat_km = model_km.predict(X_test)

In [ ]:
print('матрица ошибок :\n')
print(confusion_matrix(y_test,yhat_km))
print('\n')
print('Отчёт:\n')
print(classification_report(y_test,yhat_km))

матрица ошибок :

[[25127  5929  1865]
 [ 6655  7719  1487]
 [ 2300  1525  2219]]


Отчёт:

              precision    recall  f1-score   support

           0       0.74      0.76      0.75     32921
           1       0.51      0.49      0.50     15861
           2       0.40      0.37      0.38      6044

    accuracy                           0.64     54826
   macro avg       0.55      0.54      0.54     54826
weighted avg       0.63      0.64      0.64     54826

